In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict

import os
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

import albumentations as A
from albumentations.core.composition import Compose, OneOf
from albumentations.augmentations.transforms import CLAHE, GaussNoise, ISONoise
from albumentations.pytorch import ToTensorV2

In /home/onur/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/onur/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/onur/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/onur/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/onur/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-d

In [2]:
train_path = '../plant-pathology-2021-fgvc8/train_images_resized'
test_path = '../plant-pathology-2021-fgvc8/test_images_resized'

df = pd.read_csv('../plant-pathology-2021-fgvc8/train.csv')
df.head()

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex


In [3]:
df.shape

(18632, 2)

In [4]:
dct = defaultdict(list)

for i, label in enumerate(df.labels):
    for category in label.split():
        dct[category].append(i)

dct = {key: np.array(val) for key, val in dct.items()}
dct

{'healthy': array([    0,     5,     7, ..., 18626, 18627, 18631]),
 'scab': array([    1,     2,     3, ..., 18625, 18628, 18630]),
 'frog_eye_leaf_spot': array([    1,    14,    31, ..., 18612, 18619, 18630]),
 'complex': array([    1,     4,     8, ..., 18597, 18604, 18617]),
 'rust': array([    6,    21,    26, ..., 18601, 18616, 18629]),
 'powdery_mildew': array([   20,    39,    44, ..., 18532, 18617, 18618])}

In [5]:
new_df = pd.DataFrame(np.zeros((df.shape[0], len(dct.keys())), dtype = np.int8), columns = dct.keys())

for key, val in dct.items():
    new_df.loc[val, key] = 1

new_df.head()

,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,1,0,0,0,0,0
1,0,1,1,1,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,0,0,1,0,0


In [6]:
new_df.sum(axis = 0)

healthy               4624
scab                  5712
frog_eye_leaf_spot    4352
complex               2151
rust                  2077
powdery_mildew        1271
dtype: int64

In [7]:
cats_total = new_df.sum(axis = 1).values

np.sort(cats_total)


array([1, 1, 1, ..., 3, 3, 3])

In [8]:
np.mean(cats_total)

1.083458565908115

In [9]:
from collections import Counter

Counter(df.labels)

Counter({'healthy': 4624,
         'scab frog_eye_leaf_spot complex': 200,
         'scab': 4826,
         'complex': 1602,
         'rust': 1860,
         'frog_eye_leaf_spot': 3181,
         'powdery_mildew': 1184,
         'scab frog_eye_leaf_spot': 686,
         'frog_eye_leaf_spot complex': 165,
         'rust frog_eye_leaf_spot': 120,
         'powdery_mildew complex': 87,
         'rust complex': 97})

In [10]:
ks = {k: i for i,k in enumerate(Counter(df.labels).keys())}
new_df['labs'] = np.array([ks[v] for v in df.labels])
new_df['labs']

0        0
1        1
2        2
3        2
4        3
        ..
18627    0
18628    2
18629    4
18630    7
18631    0
Name: labs, Length: 18632, dtype: int64

In [11]:
new_df = pd.concat([df, new_df], axis = 1)

new_df.head()

,image,labels,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew,labs
0,800113bb65efe69e.jpg,healthy,1,0,0,0,0,0,0
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex,0,1,1,1,0,0,1
2,80070f7fb5e2ccaa.jpg,scab,0,1,0,0,0,0,2
3,80077517781fb94f.jpg,scab,0,1,0,0,0,0,2
4,800cbf0ff87721f8.jpg,complex,0,0,0,1,0,0,3


In [12]:
new_df.to_csv('../plant-pathology-2021-fgvc8/better_train.csv', index = False)

In [13]:
TRAIN_DIR = '../plant-pathology-2021-fgvc8/train_images_resized/'

class CFG:
    seed = 42
    model_name = 'resnet50'
    pretrained = True
    img_size = 224
    num_classes = 6
    lr = 1e-3
    min_lr = 1e-6
    t_max = 20
    num_epochs = 20
    batch_size = 16
    accum = 1
    precision = 16
    n_fold = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
class PlantDataset(Dataset):
    def __init__(self, df, transform=None):
        self.image_id = df['image'].values
        self.labels = df['labels'].values
        self.new_labels = df.iloc[:,2:-1]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_id = self.image_id[idx]
        label = self.labels[idx]
        new_label = self.new_labels.iloc[idx].values.astype(float)
        image_path = TRAIN_DIR + image_id
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        augmented = self.transform(image=image)
        image = augmented['image']
        new_label = torch.tensor(new_label, dtype=torch.float32)
        return {'image':image, 'target': new_label}

In [15]:
def get_transform(phase: str):
    if phase == 'train':
        return Compose([
            A.RandomResizedCrop(height=CFG.img_size, width=CFG.img_size),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(),
            ToTensorV2(),
        ])

In [16]:
sfk = StratifiedKFold(CFG.n_fold)
for train_idx, valid_idx in sfk.split(new_df['image'], new_df['labels']):
    df_train = new_df.iloc[train_idx]
    df_valid = new_df.iloc[valid_idx]
    break
    
print(f"train size: {len(df_train)}")
print(f"valid size: {len(df_valid)}")

train size: 14905
valid size: 3727


In [79]:
class CustomResNet(nn.Module):
    def __init__(self, model_name='resnet50', pretrained=False, requires_grad = True):
        super().__init__()
        model = models.resnet50(pretrained)
        # to freeze the hidden layers
        if requires_grad == False:
            for param in model.parameters():
                param.requires_grad = False
        # to train the hidden layers
        elif requires_grad == True:
            for param in model.parameters():
                param.requires_grad = True
        model.fc = nn.Linear(2048,6)
        self.model = model
        #in_features = model.fc.in_features
        #print(in_features)
        #self.feat_ext = nn.Sequential(*(list(model.children())[:-1])) 
        #in_features = self.model.get_classifier().in_features
        #self.model.fc = nn.Linear(in_features, CFG.num_classes)
        
    def forward(self, x):
        x = self.model(x)
        x = torch.sigmoid(x)
        return x

In [80]:
resnet = CustomResNet()

In [81]:
test = torch.randn(1,3,500,500)
resnet(test)

tensor([[0.6238, 0.7117, 0.1950, 0.4133, 0.3643, 0.3789]],
       grad_fn=<SigmoidBackward>)

In [82]:
train_dataset = PlantDataset(df_train, get_transform('train'))
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

valid_dataset = PlantDataset(df_valid, get_transform('valid'))
valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)


In [83]:
train_dataset[0]['target']

tensor([0., 0., 0., 0., 0., 1.])

In [84]:
arr = new_df.iloc[1,2:-1].values.astype(float)
torch.tensor(arr, dtype=torch.float32)

tensor([0., 1., 1., 1., 0., 0.])

In [85]:
from sklearn.metrics import accuracy_score

if torch.cuda.is_available():
    resnet.cuda()

lr = 0.001
epochs = 5
batch_size = 16
optimizer = optim.Adam(resnet.parameters(), lr=lr)
criterion = nn.BCELoss()

train_loss = []
valid_loss = []

for epoch in range(CFG.num_epochs):
    print(f"Epoch {epoch+1} of {CFG.num_epochs}")
    print('Training')
    resnet.train()
    counter = 0
    train_running_loss = 0.0
    
    correct = 0.
    total = 0.
    acc = 0.
    val_correct = 0.
    val_total = 0.

    for i, data in enumerate(train_loader):
        counter += 1
        data, target = data['image'].to(CFG.device), data['target'].to(CFG.device)
        optimizer.zero_grad()
        outputs = resnet(data)

        loss = criterion(outputs, target)
        train_running_loss += loss.item()
        # backpropagation
        loss.backward()
        # update optimizer parameters
        optimizer.step()
        
        with torch.no_grad():
            #round up and down to either 1 or 0
            predicted = np.round(outputs.cpu().detach().numpy())
            true = target.cpu().detach().numpy()
            total += (target.size(0)*target.size(1))
            acc += accuracy_score(true, predicted)
            #calculate how many images were correctly classified
            correct += (predicted == target.cpu().detach().numpy()).sum().item()
            
    accuracy = 100 * correct / total
    print("Train Accuracy: {}%".format(acc/counter*100))        

    train_epoch_loss = train_running_loss / counter

    print('Validating')
    resnet.eval()
    counter = 0
    val_running_loss = 0.0
    val_acc = 0.
    with torch.no_grad():
        for i, data in enumerate(valid_loader):
            counter += 1
            data, target = data['image'].to(CFG.device), data['target'].to(CFG.device)
            outputs = resnet(data)
            # apply sigmoid activation to get all the outputs between 0 and 1
            loss = criterion(outputs, target)
            val_running_loss += loss.item()

            predicted = np.round(outputs.cpu().detach().numpy())
            true = target.cpu().detach().numpy()
            total += (target.size(0)*target.size(1))
            val_acc += accuracy_score(true, predicted)
            #calculate how many images were correctly classified
            correct += (predicted == target.cpu().detach().numpy()).sum().item()
 
        val_accuracy = 100 * val_correct / total
        print("Val Accuracy: {}%".format(val_acc/counter*100))        

        valid_epoch_loss = val_running_loss / counter
        
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f'Val Loss: {valid_epoch_loss:.4f}')


Epoch 1 of 20
Training
Train Accuracy: 9.264232008592911%
Validating
Val Accuracy: 20.420258620689655%
Train Loss: 0.4154
Val Loss: 0.3736
Epoch 2 of 20
Training
Train Accuracy: 24.711331901181524%
Validating
Val Accuracy: 17.780172413793103%
Train Loss: 0.3624
Val Loss: 0.4109
Epoch 3 of 20
Training
Train Accuracy: 31.531954887218046%
Validating
Val Accuracy: 27.720905172413797%
Train Loss: 0.3439
Val Loss: 0.3749
Epoch 4 of 20
Training
Train Accuracy: 44.038668098818476%
Validating
Val Accuracy: 58.91702586206896%
Train Loss: 0.3111
Val Loss: 0.2483
Epoch 5 of 20
Training
Train Accuracy: 52.12137486573577%
Validating
Val Accuracy: 56.33081896551724%
Train Loss: 0.2809
Val Loss: 0.3015
Epoch 6 of 20
Training
Train Accuracy: 56.20300751879699%
Validating
Val Accuracy: 64.19719827586206%
Train Loss: 0.2629
Val Loss: 0.2147
Epoch 7 of 20
Training
Train Accuracy: 58.35794844253491%
Validating
Val Accuracy: 66.59482758620689%
Train Loss: 0.2543
Val Loss: 0.2159
Epoch 8 of 20
Training
Train

In [86]:
import cv2
def test_single_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = img / 255.
    img = np.expand_dims(img, axis=0)
    img = torch.from_numpy(img.astype(np.float32))
    img = img.permute(0,3,1,2)
    return img

In [141]:
path = "../plant-pathology-2021-fgvc8/train_images/81be3593928bb798.jpg"
img = test_single_img(path)

In [142]:
out = resnet(img.cuda())

In [143]:
np.round(out.cpu().detach().numpy())

array([[0., 0., 0., 1., 0., 0.]], dtype=float32)

In [128]:
target.size(1)

6

In [176]:
kk = 851
print(np.round(resnet(valid_dataset[kk]['image'].unsqueeze(0).cuda()).cpu().detach().numpy()))
print(valid_dataset[kk]['target'].cpu().detach().numpy())

[[0. 0. 1. 0. 0. 0.]]
[0. 0. 1. 0. 0. 0.]


In [213]:
preds = []
trues = []
for i in range(len(valid_dataset.labels)):
    y_pred = np.round(resnet(valid_dataset[i]['image'].unsqueeze(0).cuda()).cpu().detach().numpy())
    y_true = valid_dataset[i]['target'].cpu().detach().numpy()
    preds.append(y_pred[0].astype(np.int8))
    trues.append(y_true.astype(np.int8))

In [244]:
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

target_names = ['healthy', 'scab', 'frog_eye_leaf_spot', 'complex', 'rust', 'powdery_mildew']
cm = multilabel_confusion_matrix(np.asarray(trues), np.asarray(preds))
print(classification_report(np.asarray(trues), np.asarray(preds)))

              precision    recall  f1-score   support

           0       0.86      0.92      0.89       925
           1       0.92      0.70      0.80      1142
           2       0.89      0.70      0.79       870
           3       0.76      0.48      0.58       431
           4       0.80      0.93      0.86       416
           5       0.92      0.79      0.85       254

   micro avg       0.87      0.76      0.81      4038
   macro avg       0.86      0.75      0.79      4038
weighted avg       0.87      0.76      0.80      4038
 samples avg       0.80      0.79      0.79      4038



In [242]:
cm

array([[[2659,  143],
        [  72,  853]],

       [[2520,   65],
        [ 345,  797]],

       [[2781,   76],
        [ 258,  612]],

       [[3230,   66],
        [ 226,  205]],

       [[3216,   95],
        [  31,  385]],

       [[3455,   18],
        [  54,  200]]])